In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import tensorflow as tf
import tensorflow_hub as hub
from IPython.display import display, Audio
import numpy as np
from wavegan import sample_context_embeddings

In [2]:
# Create 50 random latent vectors z
_z = (np.random.randn(50, 128))

In [3]:
def generate(cond_text, inference_graph_path, model_ckpt_path, embed_model='elmo'):
    with tf.Session() as sess:
        #print([n.name for n in tf.get_default_graph().as_graph_def().node])
        if embed_model == 'elmo':
            embed = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False, name='embed')
        elif embed_model == 'nnlm':
            embed = hub.Module("https://tfhub.dev/google/nnlm-en-dim128-with-normalization/1", trainable=False, name='embed')
            
        embeddings = tf.tile(embed([cond_text]), [50, 1])

        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        _fake_conditionals = sess.run(embeddings)

    # Load the graph
    graph = tf.Graph()
    with graph.as_default():
        saver = tf.train.import_meta_graph(inference_graph_path)
        with tf.Session() as sess:
            saver.restore(sess, model_ckpt_path)

            # Synthesize G(z)
            z = graph.get_tensor_by_name('z:0')
            c = graph.get_tensor_by_name('c:0')
            G_z = graph.get_tensor_by_name('G_z:0')
            _G_z = sess.run(G_z, {z: _z, c: _fake_conditionals})
            return _G_z

In [70]:
cond_text = 'footsteps mud'

In [71]:
g1 = generate(cond_text, 'train/infer/infer.meta', 'train/model.ckpt-355610')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Restoring parameters from train/model.ckpt-355610


In [72]:
# Play audio in notebook
for i in range(0, 50):
    display(Audio(g1[i].flatten(), rate=16000))

In [ ]:
g2 = generate(cond_text, 'train_all_cond/infer/infer.meta', 'train_all_cond/model.ckpt-4855')

In [ ]:
# Play audio in notebook
for i in range(0, 50):
    display(Audio(g2[i].flatten(), rate=16000))

In [ ]:
g3 = generate(cond_text, 'train_all_cond_densenet_2/infer/infer.meta', 'train_all_cond_densenet_2/model.ckpt-58142')

In [ ]:
# Play audio in notebook
for i in range(0, 50):
    display(Audio(g3[i].flatten(), rate=16000))

In [ ]:
g4 = generate(cond_text, 'train_all_cond_pp/infer/infer.meta', 'train_all_cond_pp/model.ckpt-10833')

In [ ]:
# Play audio in notebook
for i in range(0, 50):
    display(Audio(g4[i].flatten(), rate=16000))

# Analyzing Audio Samples

In [ ]:
from scipy import fft, arange

def plot_spectrum(y, samplerate):
    """
    Plots a Single-Sided Amplitude Spectrum of y(t)
    """
    n = len(y) # length of the signal
    k = arange(n)
    T = n/samplerate
    frq = k/T # two sides frequency range
    frq = frq[range(n//2)] # one side frequency range

    Y = fft(y)/n # fft computing and normalization
    Y = Y[range(n//2)]
    
    plt.plot(frq,abs(Y),'r') # plotting the spectrum
    plt.xlabel('Freq (Hz)')
    plt.ylabel('|Y(freq)|')

i = 13
trimmed, index = librosa.effects.trim(_G_z[i].flatten(), top_db=20)
print(trimmed.shape)
print(index)
display(Audio(trimmed, rate=16000))
plt.figure(1)
#plt.axis([0, 1000, 0, 0.012])
plot_spectrum(trimmed, 16000)
plt.show()
librosa.display.waveplot(trimmed, sr=16000)


In [ ]:
# Load the graph
graph = tf.Graph()
with graph.as_default():
    saver = tf.train.import_meta_graph('train_swordfighter_pp/infer/infer.meta')
    with tf.Session() as sess:
        saver.restore(sess, 'train_swordfighter_pp/model.ckpt-19171')
        
        # Synthesize G(z)
        z = graph.get_tensor_by_name('z:0')
        G_z = graph.get_tensor_by_name('G_z:0')
        _G_z = sess.run(G_z, {z: _z})

In [ ]:
# Play audio in notebook
for i in range(0, 50):
    display(Audio(_G_z[i].flatten(), rate=16000))

In [ ]:
from scipy import fft, arange

def plot_spectrum(y, samplerate):
    """
    Plots a Single-Sided Amplitude Spectrum of y(t)
    """
    n = len(y) # length of the signal
    k = arange(n)
    T = n/samplerate
    frq = k/T # two sides frequency range
    frq = frq[range(n//2)] # one side frequency range

    Y = fft(y)/n # fft computing and normalization
    Y = Y[range(n//2)]
    
    plt.plot(frq,abs(Y),'r') # plotting the spectrum
    plt.xlabel('Freq (Hz)')
    plt.ylabel('|Y(freq)|')

i = 0
trimmed, index = librosa.effects.trim(_G_z[i].flatten(), top_db=20)
print(trimmed.shape)
print(index)
display(Audio(trimmed, rate=16000))
plt.figure(1)
#plt.axis([0, 1000, 0, 0.012])
plot_spectrum(trimmed, 16000)
plt.show()
librosa.display.waveplot(trimmed, sr=16000)